In [1]:
import cProfile
import os
import random as rd
import time
import pandas as pd
import math
from collections import defaultdict
import numpy as np
import itertools as it
import Utilities.experiment_utils as eu
import unit_test as ut
import Games.mnk as mnk
import Agents.random as arand
import Agents.vanilla_mcts as mcts 
import Agents.siea_mcts as siea_mcts
import Games.function_optimisation as fo
import Utilities.logs_management as lm
#cProfile.run("wins =  random_games(10000, base_gs)")

In [2]:

function_index = 1
runs = 3
random_seed = 1
iterations = 50
game_state = fo.GameState(function_index=function_index)
game_state.set_initial_state()
mcts_player = siea_mcts.SIEA_MCTS_Player(max_iterations=iterations, logs=True)
#mcts_player = mcts.MCTS_Player(max_iterations=iterations, logs=True)
action = eu.mcts_decision_analysis(game_state, 
                                   mcts_player, 
                                   os.path.join("Outputs","Test","Function_" + str(function_index), mcts_player.name), 
                                   runs, 
                                   random_seed)

In [2]:
#FOP experiment
random_seed = 0
runs = 30
iterations = 5000
es_lambda = 4
es_fitness_iterations = 30
es_generations = 20
c_list = [0.5, 1, math.sqrt(2),2,3]
c_names = ["0_5", "1", "1_4142", "2", "3"]
agents = []
function_indexes = []
function_indexes = [0,1,2,3,4]
function_indexes += [5,6,7,8,9]

agents = [mcts.MCTS_Player(max_iterations=iterations, 
                           logs=True, 
                           c=c, 
                           name = "MCTS_c" + c_names[i]) for i,c in enumerate(c_list)]
agents = agents + [siea_mcts.SIEA_MCTS_Player(max_iterations=its, 
                                         es_lambda=es_lambda, 
                                         es_fitness_iterations=es_fitness_iterations,
                                        es_generations=es_generations,
                                        es_semantics_l=0.1,
                                        es_semantics_u = 0.5,
                                        name = "SIEA_MCTS_its" + str(its),
                                         logs=True) for its in [iterations, iterations-(es_fitness_iterations*es_generations*es_lambda)]]
agents = agents + [siea_mcts.SIEA_MCTS_Player(max_iterations=its, 
                                         es_lambda=es_lambda, 
                                         es_fitness_iterations=es_fitness_iterations,
                                        es_generations=es_generations,
                                        name = "EA_MCTS_its" + str(its),
                                        use_semantics=False,
                                         logs=True) for its in [iterations, iterations-(es_fitness_iterations*es_generations*es_lambda)]]
                                        
for function_index in function_indexes:
    game_state = fo.GameState(function_index=function_index, n_players=1)
    game_state.set_initial_state()
    for agent in agents:
        action = eu.mcts_decision_analysis(game_state = game_state,
                                             mcts_player = agent,
                                             logs_path = os.path.join("Outputs","FO_single_decision_5000_new","Function_" + str(function_index), agent.name),
                                             runs = runs,
                                             random_seed = random_seed)                                        



In [2]:
#tested_agent.evolution_logs
#tested_agent.choose_action_logs
#print(tested_agent.view_mcts_tree())
game = mnk.GameState(11,11,5)
game.set_initial_state()
agent2 = mcts.MCTS_Player(max_iterations=500, rollouts = 50, logs=True)
agent1 = siea_mcts.SIEA_MCTS_Player(max_iterations=500, rollouts = 50, logs=True)
agent0 = arand.RandomPlayer()
game_player = eu.GamePlayer(game, [agent2, agent1])
game_player.play_games(50, logs=True)
#game_player.logs_by_action
game_player.save_data(os.path.join("Outputs","MNK_500it_50r", str(game_player)))

game_player = eu.GamePlayer(game, [agent1, agent2])
game_player.play_games(50, logs=True)
#game_player.logs_by_action
game_player.save_data(os.path.join("Outputs","MNK_500it_50r", str(game_player)))

In [ ]:
ut.run()

In [4]:

experiment_path = os.path.join("Outputs","FO_single_decision_5000_new")
file_path_list = lm.find_log_files("results.csv", experiment_path)
df = lm.combine_logs(experiment_path, "combined_results.csv", file_path_list)
df


['Outputs\\FO_single_decision_5000_new\\Function_0\\EA_MCTS_its2600\\results.csv', 'Outputs\\FO_single_decision_5000_new\\Function_0\\EA_MCTS_its5000\\results.csv', 'Outputs\\FO_single_decision_5000_new\\Function_0\\MCTS_c0_5\\results.csv', 'Outputs\\FO_single_decision_5000_new\\Function_0\\MCTS_c1\\results.csv', 'Outputs\\FO_single_decision_5000_new\\Function_0\\MCTS_c1_4142\\results.csv', 'Outputs\\FO_single_decision_5000_new\\Function_0\\MCTS_c2\\results.csv', 'Outputs\\FO_single_decision_5000_new\\Function_0\\MCTS_c3\\results.csv', 'Outputs\\FO_single_decision_5000_new\\Function_0\\SIEA_MCTS_its2600\\results.csv', 'Outputs\\FO_single_decision_5000_new\\Function_0\\SIEA_MCTS_its5000\\results.csv', 'Outputs\\FO_single_decision_5000_new\\Function_1\\EA_MCTS_its2600\\results.csv', 'Outputs\\FO_single_decision_5000_new\\Function_1\\EA_MCTS_its5000\\results.csv', 'Outputs\\FO_single_decision_5000_new\\Function_1\\MCTS_c0_5\\results.csv', 'Outputs\\FO_single_decision_5000_new\\Function_1\

,avg_eval_of_max_reward_leaf_node,std_eval_of_max_reward_leaf_node,avg_eval_of_max_visits_leaf_node,std_eval_of_max_visits_leaf_node,avg_terminals_count,std_terminals_count,avg_nodes_by_iteration,std_nodes_by_iteration,avg_forward_model_calls,std_forward_model_calls,avg_time,std_time,avg_root_avg_reward,std_root_avg_reward,random_seed,runs,Path_origin
0,0.735827,0.321952,0.735827,0.321952,3.800000,8.260417,0.114308,0.298836,41184.000000,5921.778665,1.215891,0.263961,0.687694,0.218948,1,30,Outputs\FO_single_decision_5000_new\Function_0...
1,0.715196,0.358689,0.715196,0.358689,4.066667,8.452436,0.126540,0.311137,43610.100000,11516.558647,1.738447,0.514099,0.746788,0.244122,1,30,Outputs\FO_single_decision_5000_new\Function_0...
2,0.998545,0.001422,0.998545,0.001422,0.000000,0.000000,1.000000,0.000000,31283.300000,233.811912,0.925610,0.048214,0.953340,0.002242,1,30,Outputs\FO_single_decision_5000_new\Function_0...
3,0.998966,0.001335,0.998966,0.001335,0.000000,0.000000,1.000000,0.000000,31209.033333,253.603884,0.925229,0.075860,0.953753,0.001947,1,30,Outputs\FO_single_decision_5000_new\Function_0...
4,0.998810,0.001167,0.998810,0.001167,0.000000,0.000000,1.000000,0.000000,31129.933333,231.095880,0.908567,0.089566,0.953427,0.001905,1,30,Outputs\FO_single_decision_5000_new\Function_0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0.556738,0.118776,0.556738,0.118776,0.000000,0.000000,1.000000,0.000000,57565.066667,1674.355026,1.505584,0.230548,0.200933,0.033815,1,30,Outputs\FO_single_decision_5000_new\Function_9...
86,0.525244,0.153007,0.525244,0.153007,0.000000,0.000000,1.000000,0.000000,57493.733333,1724.273519,1.494345,0.210272,0.203120,0.033242,1,30,Outputs\FO_single_decision_5000_new\Function_9...
87,0.624767,0.052813,0.624767,0.052813,0.000000,0.000000,1.000000,0.000000,56977.933333,1777.440312,1.512348,0.221972,0.211440,0.033426,1,30,Outputs\FO_single_decision_5000_new\Function_9...
88,0.196541,0.149481,0.196541,0.149481,12.800000,37.888584,0.089821,0.183376,41167.666667,5846.841218,2.024176,0.475387,0.314433,0.253335,1,30,Outputs\FO_single_decision_5000_new\Function_9...
